In [1]:
import cv2
import numpy as np
import datetime

In [2]:
# Define color ranges for ball detection (in HSV color space)
color_ranges = {
        'Red': ((0, 100, 100), (10, 255, 255)),
        'Green': ((40, 100, 100), (70, 255, 255)),
        'Blue': ((100, 100, 100), (130, 255, 255)),
        'Yellow': ((20, 100, 100), (30, 255, 255)),
        'White': ((0, 0, 200), (180, 30, 255))
    }

In [3]:
# Load video
video_path = 'AI.mp4'
cap = cv2.VideoCapture(video_path)

In [4]:
# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
num_quadrants_x = 2
num_quadrants_y = 2
quad_width = frame_width // num_quadrants_x
quad_height = frame_height // num_quadrants_y

In [5]:
# Initialize variables
tracked_objects = {}
event_log = []
output_video_path = 'output_video.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

In [6]:
def get_quadrant(x, y, width, height):
    quad_x = x // quad_width + 1
    quad_y = y // quad_height + 1
    return (quad_y - 1) * num_quadrants_x + quad_x

In [7]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0  # Timestamp in seconds
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    for color_name, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            if cv2.contourArea(contour) > 500:
                x, y, w, h = cv2.boundingRect(contour)
                cx, cy = x + w // 2, y + h // 2
                quad = get_quadrant(cx, cy, frame_width, frame_height)
                
                if color_name not in tracked_objects:
                    tracked_objects[color_name] = {'position': (cx, cy), 'quadrant': quad}
                    event_log.append(f"{timestamp}, {quad}, {color_name}, Entry")
                else:
                    prev_quad = tracked_objects[color_name]['quadrant']
                    if quad != prev_quad:
                        event_log.append(f"{timestamp}, {prev_quad}, {color_name}, Exit")
                        event_log.append(f"{timestamp}, {quad}, {color_name}, Entry")
                        tracked_objects[color_name]['quadrant'] = quad
                cv2.circle(frame, (cx, cy), 10, (0, 255, 0), -1)
                cv2.putText(frame, f"{color_name} - {quad}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    out.write(frame)

cap.release()
out.release()

# Write event log to file
with open('event_log.txt', 'w') as f:
    for event in event_log:
        f.write(event + '\n')

print("Processing complete. Output saved.")


Processing complete. Output saved.
